In [1]:
!pip install tqdm boto3 requests regex -q

     |████████████████████████████████| 655kB 4.9MB/s 


In [2]:
import torch
from IPython.display import clear_output

GITHUB_REPO = "huggingface/pytorch-pretrained-BERT" # 感謝 HuggingFace 團隊造福後人
PRETRAINED_MODEL_NAME = "bert-base-chinese"  # 指定繁簡中文 BERT-BASE 預訓練模型

# 取得此預訓練模型所使用的 tokenizer
tokenizer = torch.hub.load(GITHUB_REPO, 'bertTokenizer', PRETRAINED_MODEL_NAME)

clear_output()
print("PyTorch 版本：", torch.__version__)

PyTorch 版本： 1.1.0


In [3]:
vocab = tokenizer.vocab
print("字典大小：", len(vocab))

字典大小： 21128


In [4]:
import random
random_tokens = random.sample(list(vocab), 10)
random_ids = [vocab[t] for t in random_tokens]

print("{0:20}{1:15}".format("token", "index"))
print("-" * 25)
for t, id in zip(random_tokens, random_ids):
    print("{0:15}{1:10}".format(t, id))

token               index          
-------------------------
蕎                    5935
##緞                 18280
‖                     337
##ς                 13395
##則                 14236
谷                    6484
266                  9674
彤                    2502
##箕                 18106
##mate              12125


In [5]:
indices = list(range(647, 657))
some_pairs = [(t, idx) for t, idx in vocab.items() if idx in indices]
for pair in some_pairs:
    print(pair)

('ㄅ', 647)
('ㄆ', 648)
('ㄇ', 649)
('ㄉ', 650)
('ㄋ', 651)
('ㄌ', 652)
('ㄍ', 653)
('ㄎ', 654)
('ㄏ', 655)
('ㄒ', 656)


In [6]:
text = "[CLS] 等到潮水 [MASK] 了，就知道誰沒穿褲子。"
tokens = tokenizer.tokenize(text)
ids = tokenizer.convert_tokens_to_ids(tokens)

print(text)
print(tokens[:10], '...')
print(ids[:10], '...')

[CLS] 等到潮水 [MASK] 了，就知道誰沒穿褲子。
['[CLS]', '等', '到', '潮', '水', '[MASK]', '了', '，', '就', '知'] ...
[101, 5023, 1168, 4060, 3717, 103, 749, 8024, 2218, 4761] ...


In [6]:
maskedLM_model = torch.hub.load(GITHUB_REPO, 
                                'bertForMaskedLM', 
                                PRETRAINED_MODEL_NAME)

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-pretrained-BERT_master


In [147]:
masked = '[MASK]'
#text = "等到潮水退了，就知道誰沒穿褲子。"
text = input("輸入句子： \n")
text = ''.join(text.split())
for j in range(0,len(text)):
    if text[j] in ['，','。']:
      continue
    mask_text = text.replace(text[j:j+1],masked)
    mask_text = '[CLS] ' + mask_text
    #print(mask_text)
    tokens = tokenizer.tokenize(mask_text)
    ids = tokenizer.convert_tokens_to_ids(tokens)
    #print(tokens)
    #print(ids)
    tokens_tensor = torch.tensor([ids])  # (1, seq_len)
    segments_tensors = torch.zeros_like(tokens_tensor)  # (1, seq_len)
    #maskedLM_model = torch.hub.load(GITHUB_REPO, 
    #                            'bertForMaskedLM', 
    #                            PRETRAINED_MODEL_NAME)
    
    maskedLM_model.eval()
    with torch.no_grad():
        outputs = maskedLM_model(tokens_tensor, segments_tensors)
        predictions = outputs[0]
    # (1, seq_len, num_hidden_units)
    #del maskedLM_model
    
    masked_index = j+1
    #masked_index = 5
    k = 5
    probs, indices = torch.topk(torch.softmax(predictions[0, masked_index], -1), k)
    predicted_tokens = tokenizer.convert_ids_to_tokens(indices.tolist())
    predicted_tokens_prob= []
    #print(predicted_tokens)
    
    for i, (t, p) in enumerate(zip(predicted_tokens, probs), 1):
        tokens[masked_index] = t
        prob = "{:2}%".format(int(p.item()*100))
        predicted_tokens_prob.append( prob )
    
    if(text[j] not in predicted_tokens):
        detect_sentence =  text[:j] +"\033[35m "+text[j]+" \033[0m" +text[j+1:]
        print('錯字預測:',detect_sentence)
        candid = zip(predicted_tokens[:5],predicted_tokens_prob[:5])
        print('建議字:', list(candid))

輸入句子： 
等到潮水退了，就知道誰沒穿褲子。
錯字預測: 等到潮水 退 了，就知道誰沒穿褲子。
建議字: [('來', '67%'), ('濕', '25%'), ('過', ' 2%'), ('流', ' 0%'), ('走', ' 0%')]
錯字預測: 等到潮水退了，就知道 誰 沒穿褲子。
建議字: [('我', '53%'), ('你', '12%'), ('他', '11%'), ('她', ' 7%'), ('還', ' 5%')]
